In [1]:
import sentence_transformers
import torch
import pandas as pd
import numpy as np

In [4]:
#SP_ESG_350 = pd.read_excel("PerformanceGraphExport.xls" ,index_col = 0)[5:]

In [5]:
esg_documents_df = pd.read_csv("esg_documents_for_dax_companies.csv", sep="|")
esg_documents_df.head()

,Unnamed: 0,company,content,datatype,date,domain,esg_topics,internal,symbol,title,url
0,2,Beiersdorf AG,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,2021-03-31,NaN,"['CleanWater', 'GHGEmission', 'ProductLiabilit...",1,BEI,BeiersdorfAG Sustainability Report 2021,NaN
1,3,Deutsche Telekom AG,Corporate Responsibility Report 2021 2 Content...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'Iso50001', 'GlobalWarming', ...",1,DTE,DeutscheTelekomAG Sustainability Report 2021,NaN
2,5,Vonovia SE,VONOVIA SE SUSTAINABILITY REPORT 2021 =For a S...,sustainability_report,2021-03-31,NaN,"['Whistleblowing', 'DataSecurity', 'Vaccine', ...",1,VNA,VonoviaSE Sustainability Report 2021,NaN
3,6,Merck KGaA,Sustainability Report 2021 TABLE OF CONTENTS S...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'DataMisuse', 'DrugResistance...",1,MRK,MerckKGaA Sustainability Report 2021,NaN
4,9,MTU,Our ideas and concepts FOR A SUSTAINABLE FUTUR...,sustainability_report,2020-03-31,NaN,"['WorkLifeBalance', 'Corruption', 'AirQuality'...",1,MTX,MTUAeroEngines Sustainability Report 2020,NaN


In [6]:
esg_documents_df["esg_topics"]

0        ['CleanWater', 'GHGEmission', 'ProductLiabilit...
1        ['DataSecurity', 'Iso50001', 'GlobalWarming', ...
2        ['Whistleblowing', 'DataSecurity', 'Vaccine', ...
3        ['DataSecurity', 'DataMisuse', 'DrugResistance...
4        ['WorkLifeBalance', 'Corruption', 'AirQuality'...
                               ...                        
11543    ['CustomerService', 'Meat', 'HumanCapital', 'T...
11544                                  ['WasteManagement']
11545    ['CustomerService', 'Recruiting', 'Sustainable...
11546    ['ClimateTech', 'Cybersecurity', 'CorporateCul...
11547    ['Renumeration', 'Recruiting', 'HumanCapital',...
Name: esg_topics, Length: 11548, dtype: object

In [7]:
# form texts by concatenating title and content
esg_texts = esg_documents_df.apply(lambda row: " ".join([str(row["title"]), str(row["content"])]), axis=1)
esg_texts[0][:100]

'BeiersdorfAG Sustainability Report 2021 Sustainability Highlight Report CARE BEYOND SKIN 2021 03 For'

In [8]:
sdg_df = pd.read_csv("sdg_descriptions_with_targetsText.csv")
sdg_df

,id,name,description,targets,targets_json_array,progress
0,1,No Poverty,End poverty in all its forms everywhere,"['1.1', 'By 2030, eradicate extreme poverty fo...","[{""target"":""1.1"",""description"":""By 2030, eradi...",['The impact of the COVID-19 pandemic reversed...
1,2,Zero Hunger,"End hunger, achieve food security and improved...","['2.1', 'By 2030, end hunger and ensure access...","[{""target"":""2.1"",""description"":""By 2030, end h...","['Between 2014 and the onset of the pandemic, ..."
2,3,Good Health and Well-being,Ensure healthy lives and promote well-being fo...,"['3.1', 'By 2030, reduce the global maternal m...","[{""target"":""3.1"",""description"":""By 2030, reduc...","['By April 2022, the coronavirus causing COVID..."
3,4,Quality Education,Ensure inclusive and equitable quality educati...,"['4.1', 'By 2030, ensure that all girls and bo...","[{""target"":""4.1"",""description"":""By 2030, ensur...",['The COVID-19 outbreak has caused a global ed...
4,5,Gender Equality,Achieve gender equality and empower all women ...,"['5.1', 'End all forms of discrimination again...","[{""target"":""5.1"",""description"":""End all forms ...","[""The world is not on track to achieve gender ..."
5,6,Clean Water and Sanitation,Ensure availability and sustainable management...,"['6.1', 'By 2030, achieve universal and equita...","[{""target"":""6.1"",""description"":""By 2030, achie...","['Access to safe water, sanitation and hygiene..."
6,7,Affordable and Clean Energy,"Ensure access to affordable, reliable, sustain...","['7.1', 'By 2030, ensure universal access to a...","[{""target"":""7.1"",""description"":""By 2030, ensur...","['Despite progress, there are still over 700 m..."
7,8,Decent Work and Economic Growth,"Promote sustained, inclusive and sustainable e...","['8.1', 'Sustain per capita economic growth in...","[{""target"":""8.1"",""description"":""Sustain per ca...","['In 2020, the COVID-19 pandemic unleashed the..."
8,9,"Industry, Innovation and Infrastructure","Build resilient infrastructure, promote inclus...","['9.1', 'Develop quality, reliable, sustainabl...","[{""target"":""9.1"",""description"":""Develop qualit...",['The manufacturing industry exhibited a recov...
9,10,Reduced Inequalities,Reduce inequality within and among countries,"['10.1', 'By 2030, progressively achieve and s...","[{""target"":""10.1"",""description"":""By 2030, prog...",['The COVID-19 crisis has exacerbated global i...


In [9]:
new_names = [7,9,11,12,13,14,15]

sdg_climate_df = sdg_df[sdg_df["name"].isin(new_names)]
sdg_climate_df.head()

,id,name,description,targets,targets_json_array,progress
12,13,Climate Action,Take urgent action to combat climate change an...,"['13.1', 'Strengthen resilience and adaptive c...","[{""target"":""13.1"",""description"":""Strengthen re...","['The increases in heatwaves, droughts and flo..."


In [10]:
sdg_texts = sdg_df.apply(lambda row: " ".join([row["name"], row["description"], row["targets"], row["progress"]]), axis=1)
sdg_climate_texts = sdg_climate_df.apply(lambda row: " ".join([row["name"], row["description"], row["targets"], row["progress"]]), axis=1)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = sentence_transformers.SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [13]:
# company_text_embeddings = retriever.encode(esg_texts)
# sdg_embeddings = retriever.encode(sdg_climate_texts)

In [14]:
def cosine_similarity(embedding1, embedding2):
    sim = np.dot(embedding1, embedding2)/(np.linalg.norm(embedding1)*np.linalg.norm(embedding2))
    return sim

In [15]:
all_companies_embedding = np.mean(company_text_embeddings, axis=0)

NameError: name 'company_text_embeddings' is not defined

In [16]:
sdg_relevance_scores = [cosine_similarity(all_companies_embedding, sdg_embedding) for sdg_embedding in sdg_embeddings]
sdg_relevance_series = pd.Series(sdg_relevance_scores, index=sdg_df["name"])
sdg_relevance_series.sort_values(inplace=True)
sdg_relevance_series.head()

NameError: name 'sdg_embeddings' is not defined

In [ ]:
sgd_relevence_series.columns

In [ ]:
sorted(companies, key=lambda company: np.sum(heatmap_array[:, companies.index(company)]))